In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [6]:
def box_count(image, box_size):
    """Count the number of boxes needed to cover the image."""
    count = 0
    for y in range(0, image.shape[0], box_size):
        for x in range(0, image.shape[1], box_size):
            # Check if there is any non-zero pixel in the box
            if np.any(image[y:y+box_size, x:x+box_size] > 0):
                count += 1
    return count

def fractal_dimension(image, max_box_size=32, min_box_size=2):
    """Calculate the fractal dimension using box-counting method."""
    # Convert image to grayscale (if needed)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    
    # Apply a binary threshold to ensure we're working with clear features
    _, gray_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

    # Calculate the box counts for multiple box sizes
    box_sizes = []
    counts = []
    for box_size in range(min_box_size, max_box_size + 1):
        count = box_count(gray_image, box_size)
        if count > 0:
            counts.append(count)
            box_sizes.append(1 / box_size)

    if len(counts) < 2:  # Not enough data for log-log fit
        return None
    
    # Compute the fractal dimension using log-log plot
    log_box_sizes = np.log(box_sizes)
    log_counts = np.log(counts)
    
    # Linear regression to estimate the slope (fractal dimension)
    slope, intercept = np.polyfit(log_box_sizes, log_counts, 1)
    
    return -slope  # Negative slope is the fractal dimension

In [9]:
def calculate_entropy(image):
    """Calculate the Shannon entropy of an image."""
    # Flatten the image to a 1D array and compute histogram
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist /= hist.sum()  # Normalize histogram to get probabilities
    entropy = -np.sum(hist * np.log2(hist + 1e-7))  # Add small epsilon to avoid log(0)
    return entropy


In [12]:
# Example usage on an image
image = cv2.imread('/home/oem/eliza/DL/project/data/artifact/real/a-y-jackson_october-morning-algoma-wartz-lake-1920.jpg')

# Convert image to LAB color space
lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

# Calculate fractal dimension for each channel (L, A, B)
l_channel, a_channel, b_channel = cv2.split(lab_image)

# Fractal dimension for each channel
fd_l = fractal_dimension(l_channel)
fd_a = fractal_dimension(a_channel)
fd_b = fractal_dimension(b_channel)

print(f"Fractal Dimension of L channel: {fd_l}")
print(f"Fractal Dimension of A channel: {fd_a}")
print(f"Fractal Dimension of B channel: {fd_b}")

Fractal Dimension of L channel: -1.9412795768128503
Fractal Dimension of A channel: -1.9412795768128503
Fractal Dimension of B channel: -1.9412795768128503


In [13]:
# Shannon entropy for each channel
entropy_l = calculate_entropy(l_channel)
entropy_a = calculate_entropy(a_channel)
entropy_b = calculate_entropy(b_channel)

print(f"Shannon Entropy of L channel: {entropy_l}")
print(f"Shannon Entropy of A channel: {entropy_a}")
print(f"Shannon Entropy of B channel: {entropy_b}")

Shannon Entropy of L channel: 7.1472039222717285
Shannon Entropy of A channel: 4.952820777893066
Shannon Entropy of B channel: 5.406389236450195
